---
title: "Exercicio 2"
subtitle: "Mineração de Dados"
author: "Caio Rubem Saboia Monteiro"
# date: "dd/mm/aaaa" #if necessary
format:
  html:
    toc: true
    number-sections: true
    # toc-location: right-body
    code-fold: false
    # css: styles.css #if necessary
    math: mathjax

  pdf:
    number-sections: true
    toc: true
    keep_tex: true

latex-engine: xelatex

header-includes:
  - \usepackage{amsmath}
  - \usepackage{amsfonts}
  - \usepackage{amssymb}
  - \usepackage{listings}

execute:
  cache: true
  enabled: true
  freeze: true #can be use 'false' or auto
  echo: true
  # daemon: false #default is 300, but can use boleean values too 

#python
jupyter: python3 #can be use for Julia too
  # or can be use something like this:
# kernelspec:
#     name: "xpython"
#     language: "python"
#     display_name: "Python 3.7 (XPython)"

#R
# knitr:
#   opts_chunk:
#     collapse: true
#     comment: ">>"
#     R.options:
#       knitr.graphics.auto_pdf: true

# engine: julia # for more aplicatoins use quarto.org or :QuartoHelp Julia
---


Aplicando modelos avançados de aprendizado supervisionado


# MNIST - Reconhecimento de algarismos escritos a mão


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import random

# 1. Carregar o conjunto de dados MNIST
data = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = data.load_data()

# 2. Normalizar os dados (0-255 -> 0-1)
x_train, x_test = x_train / 255.0, x_test / 255.0

# 3. Definir a arquitetura da rede neural
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),  # Achatar imagens 28x28 para vetor 784
    keras.layers.Dense(70, activation='relu'),  # Camada oculta com 128 neurônios
    keras.layers.Dense(10, activation='softmax') # Camada de saída para 10 classes
])

# 4. Compilar o modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# 5. Treinar o modelo
epochs = 20  # Número de épocas
history = model.fit(x_train, y_train, epochs=epochs, validation_data=(x_test, y_test))

# 6. Avaliação no conjunto de teste
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Perda: {loss:.4f}, Acurácia: {accuracy:.4f}')

In [ ]:
# 7. Exemplo de previsão
predictions = model.predict(x_test)
plt.imshow(x_test[0], cmap=plt.cm.binary)
plt.title(f'Predição: {np.argmax(predictions[0])}')
plt.show()

# 8. Visualizar erros do modelo
# 9. Sortear várias imagens aleatórias e verificar as predições
random_indices = random.sample(range(len(x_test)), 10)
predictions_list = [np.argmax(predictions[idx]) for idx in random_indices]
real_values_list = [y_test[idx] for idx in random_indices]

for i, idx in enumerate(random_indices):
    if predictions_list[i] == real_values_list[i]:
        print(f"acertou")
    else:
        print(f"errou")

# Comentários da Google Store


In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Carregar o CSV
df = pd.read_csv('data/googlestore.csv')  # Ajuste o caminho se necessário

# Verificar estrutura dos dados
print(df.head())

In [ ]:
nltk.download('stopwords')
nltk.download('rslp')

stop_words = stopwords.words('portuguese')
stemmer = RSLPStemmer()

def preprocess_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-záéíóúâêîôûãõç\s]', '', text)
    tokens = text.split()
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

df['clean_text'] = df['content'].apply(preprocess_text)

df = df[df['clean_text'].str.strip() != '']

In [ ]:
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['clean_text']).toarray()
y = df['sentiment_polarity']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

print("Relatório de Classificação:")
print(classification_report(y_test, y_pred))

# Gerar a matriz de confusão
cm = confusion_matrix(y_test, y_pred)

# Obter as classes únicas (para os rótulos)
unique_labels = sorted(df['sentiment_polarity'].unique())

# Plotar a matriz de confusão
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Matriz de Confusão', fontsize=14)
plt.colorbar()

# Adicionar rótulos
tick_marks = np.arange(len(unique_labels))
plt.xticks(tick_marks, unique_labels, rotation=45)
plt.yticks(tick_marks, unique_labels)

# Preencher com os valores
thresh = cm.max() / 2.0
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'),
                 ha="center", va="center",
                 color="white" if cm[i, j] > thresh else "black")

plt.ylabel('Rótulo Verdadeiro', fontsize=12)
plt.xlabel('Previsão do Modelo', fontsize=12)
plt.tight_layout()
plt.show()